# 5.2 基本功能

# 重新索引

In [1]:
import numpy as np
import pandas as pd 

In [2]:
###pandas对象的一个重要方法是reindex，其作用是创建一个新对象，它的数据符合新的索引。看下面的例子：
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])

In [3]:
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [4]:
#用该Series的reindex将会根据新索引进行重排。如果某个索引值当前不存在，就引入缺失值：
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])

In [5]:
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [6]:
#对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理。method选项即可达到此目的，例如，使用ffill可以实现前向值填充：
obj3 = pd.Series(['blue', 'purple', 'yellow'],index=[0,2,4])

In [7]:
obj3

0      blue
2    purple
4    yellow
dtype: object

In [8]:
obj3.reindex(range(6),method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [9]:
###借助DataFrame，reindex可以修改（行）索引和列。只传递一个序列时，会重新索引结果的行：

In [10]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                   index=['a', 'c', 'd'],
                  columns=['Ohio', 'Texas', 'California'])

In [11]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [12]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])

In [13]:
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [14]:
#列可以用columns关键字重新索引：
states = ['Texas', 'Utah', 'California']

In [15]:
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


# 丢弃指定轴上的项

In [16]:
#丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。由于需要执行一些数据整理和集合逻辑，所以drop方法返回的是一个在指定轴上删除了指定值的新对象：

In [17]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])

In [18]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [19]:
new_obj = obj.drop('c')

In [20]:
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [21]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [22]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [23]:
###对于DataFrame，可以删除任意轴上的索引值。为了演示，先新建一个DataFrame例子：
data  = pd.DataFrame(np.arange(16).reshape(4,4),
                     index=['Ohio', 'Colorado', 'Utah', 'New York'],
                     columns=['one', 'two', 'three', 'four'])

In [24]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [25]:
#用标签序列调用drop会从行标签（axis 0）删除值：(注意跟数组axis=0是求列的计算区别)
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [26]:
###通过传递axis=1或axis='columns'可以删除列的值：
data.drop('two',axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [27]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


In [28]:
#许多函数，如drop，会修改Series或DataFrame的大小或形状，可以就地修改对象，不会返回新的对象：
obj.drop('c', inplace=True)

In [29]:
obj
#小心使用inplace，它会销毁所有被删除的数据。

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

# 索引、选取和过滤

In [30]:
#Series索引（obj[...]）的工作方式类似于NumPy数组的索引，只不过Series的索引值不只是整数。下面是几个例子：

In [31]:
obj = pd.Series(np.arange(4.),index=['a', 'b', 'c', 'd'])

In [32]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [33]:
obj['b']

1.0

In [34]:
obj[1]

1.0

In [35]:
 obj[2:4]

c    2.0
d    3.0
dtype: float64

In [36]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [37]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [38]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

In [39]:
###利用标签的切片运算与普通的Python切片运算不同，其末端是包含的：
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [40]:
#用切片可以对Series的相应部分进行设置：
obj['b':'c'] = 5

In [41]:
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

In [42]:
###用一个值或序列对DataFrame进行索引其实就是获取一个或多个列：

In [43]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
          index=['Ohio', 'Colorado', 'Utah', 'New York'],
                  columns=['one', 'two', 'three', 'four'])

In [44]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [45]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [46]:
data[['three','one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [47]:
#这种索引方式有几个特殊的情况。首先通过切片或布尔型数组选取数据：

In [48]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [49]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [50]:
###选取行的语法data[:2]十分方便。向[ ]传递单一的元素或列表，就可选择列。

###另一种用法是通过布尔型DataFrame（比如下面这个由标量比较运算得出的）进行索引：

In [51]:
data < 5 

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [52]:
data[data < 5] = 0

In [53]:
data
#这使得DataFrame的语法与NumPy二维数组的语法很像。

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


# 用loc和iloc进行选取

In [54]:
###对于DataFrame的行的标签索引，我引入了特殊的标签运算符loc和iloc。它们可以让你用类似NumPy的标记，使用轴标签（loc）或整数索引（iloc），从DataFrame选择行和列的子集。
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [55]:
###作为一个初步示例，让我们通过标签选择一行和多列：
data.loc['Colorado', ['two', 'three']]
#使用.loc[ ]第一个参数是行索引，第二个参数是列索引

two      5
three    6
Name: Colorado, dtype: int32

In [56]:
#然后用iloc和整数进行选取：
data.iloc[2, [3, 0, 1]]
#第一个参数是行索引，第二个参数是列索引

four    11
one      8
two      9
Name: Utah, dtype: int32

In [57]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [58]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [59]:
###这两个索引函数也适用于一个标签或多个标签的切片：
data.loc[:'Utah', 'two']
# ,前从表示第0行一直到'Utah'行，后面表示'two'这一列

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [60]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


In [61]:
###总结
###对于loc选取行列数据：

#行根据行标签，也就是索引筛选，列根据列标签，列名筛选
#如果选取的是所有行或者所有列，可以用：代替
#行标签选取的时候，两端都包含，比如[0:5]指的是0，1，2，3，4，5

###对于iloc选取行列数据：
#iloc基于位置索引，简言之，就是第几行第几列，只不过这里的行列都是从0开始的。
#iloc的0：X中不包括X，只能到X-1.

# 整数索引

In [62]:
###处理整数索引的pandas对象常常难住新手，因为它与Python内置的列表和元组的索引语法不同。例如，你可能不认为下面的代码会出错：
ser = pd.Series(np.arange(3.))

In [64]:
ser#这里，pandas可以勉强进行整数索引，但是会导致小bug。我们有包含0,1,2的索引，但是引入用户想要的东西（基于标签或位置的索引）很难：
#ser[-1]错误的

0    0.0
1    1.0
2    2.0
dtype: float64

In [65]:
#另外，对于非整数索引，不会产生歧义：
ser2 = pd.Series(np.arange(3.),index=['a', 'b', 'c'])

In [67]:
ser2[-1]

2.0

In [68]:
###为了进行统一，如果轴索引含有整数，数据选取总会使用标签。为了更准确，请使用loc（标签）或iloc（整数）：
ser[:1]

0    0.0
dtype: float64

In [69]:
ser.loc[:1]

0    0.0
1    1.0
dtype: float64

In [70]:
ser.iloc[:1]

0    0.0
dtype: float64

# 算术运算和数据对齐

In [71]:
#pandas最重要的一个功能是，它可以对不同索引的对象进行算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。对于有数据库经验的用户，这就像在索引标签上进行自动外连接。看一个简单的例子：

In [72]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])

In [73]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
              index=['a', 'c', 'e', 'f', 'g'])

In [74]:
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [75]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [76]:
s1 + s2
#自动的数据对齐操作在不重叠的索引处引入了NA值。缺失值会在算术运算过程中传播。

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [77]:
#对于DataFrame，对齐操作会同时发生在行和列上：

In [78]:
df1 = pd.DataFrame(np.arange(9.).reshape(3,3),columns=list('bcd'),
                  index=['Ohio', 'Texas', 'Colorado'])

In [82]:
df2 = pd.DataFrame(np.arange(12.).reshape(4,3),columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [83]:
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [84]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [85]:
#把它们相加后将会返回一个新的DataFrame，其索引和列为原来那两个DataFrame的并集：
df1 + df2
###因为'c'和'e'列均不在两个DataFrame对象中，在结果中以缺省值呈现。行也是同样。

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


In [86]:
#如果DataFrame对象相加，没有共用的列或行标签，结果都会是空：

In [87]:
df1 = pd.DataFrame({'A': [1, 2]})

In [88]:
df2 = pd.DataFrame({'B': [3, 4]})

In [89]:
df1

,A
0,1
1,2


In [90]:
df2

,B
0,3
1,4


In [91]:
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN


# 在算术方法中填充值

In [92]:
#在对不同索引的对象进行算术运算时，你可能希望当一个对象中某个轴标签在另一个对象中找不到时填充一个特殊值（比如0）：

In [94]:
df1 = pd.DataFrame(np.arange(12.).reshape((3,4)),columns=list('abcd'))

In [95]:
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),columns=list('abcde'))

In [96]:
df2.loc[1,'b'] = np.nan

In [97]:
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [98]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [99]:
#将它们相加时，没有重叠的位置就会产生NA值：
df1 + df2
###注意5+NaN也是缺失值

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [100]:
#使用df1的add方法，传入df2以及一个fill_value参数：
df1.add(df2,fill_value=0)
#会在加之前 填充规则：行列组合，有一个值就填充，都没有的就不填充

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [101]:
###它们每个都有一个副本，以字母r开头，它会翻转参数。因此这两个语句是等价的：
1 / df1

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


In [102]:
df1.rdiv(1)

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


In [103]:
#与此类似，在对Series或DataFrame重新索引时，也可以指定一个填充值：
df1.reindex(columns=df2.columns, fill_value=0)
#用0代替缺失值

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


# DataFrame和Series之间的运算

In [104]:
#跟不同维度的NumPy数组一样，DataFrame和Series之间算术运算也是有明确规定的。先来看一个具有启发性的例子，计算一个二维数组与其某行之间的差：

In [105]:
arr = np.arange(12.).reshape((3,4))

In [106]:
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [107]:
arr[0]

array([0., 1., 2., 3.])

In [108]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

In [109]:
#当我们从arr减去arr[0]，每一行都会执行这个操作。这就叫做广播（broadcasting），附录A将对此进行详细讲解。DataFrame和Series之间的运算差不多也是如此：

In [110]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
       columns=list('bde'),
     index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [111]:
series = frame.iloc[0]

In [113]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [114]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [115]:
#默认情况下，DataFrame和Series之间的算术运算会将Series的索引匹配到DataFrame的列，然后沿着行一直向下广播：
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [116]:
#如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集：

In [117]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])

In [118]:
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


In [119]:
#如果你希望匹配行且在列上广播，则必须使用算术运算方法。例如：
series3 = frame['d']

In [120]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [121]:
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [122]:
frame.sub(series3,axis='index')
#传入的轴号就是希望匹配的轴。在本例中，我们的目的是匹配DataFrame的行索引（axis='index' or axis=0）并进行广播。

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


# 函数应用和映射

In [123]:
#NumPy的ufuncs（元素级数组方法）也可用于操作pandas对象：

In [124]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
       index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [125]:
frame

,b,d,e
Utah,0.431102,0.215389,-0.030096
Ohio,1.001505,-0.406169,0.176440
Texas,-1.956753,-0.266467,0.465382
Oregon,1.266384,-0.472231,-0.214969


In [126]:
np.abs(frame)

,b,d,e
Utah,0.431102,0.215389,0.030096
Ohio,1.001505,0.406169,0.176440
Texas,1.956753,0.266467,0.465382
Oregon,1.266384,0.472231,0.214969


In [127]:
#另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上。DataFrame的apply方法即可实现此功能：


In [128]:
f = lambda x: x.max() - x.min()
###lambda与def的区别：

#1）def创建的方法是有名称的，而lambda没有。

#2）lambda会返回一个函数对象，但这个对象不会赋给一个标识符，而def则会把函数对象赋值给一个变量（函数名）。

#3）lambda只是一个表达式，而def则是一个语句。

#4）lambda表达式” : “后面，只能有一个表达式，def则可以有多个。

#5）像if或for或print等语句不能用于lambda中，def可以。

#6）lambda一般用来定义简单的函数，而def可以定义复杂的函数。


In [129]:
frame.apply(f)
#这里的函数f，计算了一个Series的最大值和最小值的差，在frame的每列都执行了一次。结果是一个Series，使用frame的列作为索引

b    3.223137
d    0.687620
e    0.680351
dtype: float64

In [130]:
##如果传递axis='columns'到apply，这个函数会在每行执行：
frame.apply(f, axis='columns')

Utah      0.461198
Ohio      1.407675
Texas     2.422135
Oregon    1.738615
dtype: float64

In [131]:
#许多最为常见的数组统计功能都被实现成DataFrame的方法（如sum和mean），因此无需使用apply方法。

In [136]:
#传递到apply的函数不是必须返回一个标量，还可以返回由多个值组成的Series：
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

In [137]:
frame.apply(f)

,b,d,e
min,-1.956753,-0.472231,-0.214969
max,1.266384,0.215389,0.465382


In [138]:
#元素级的Python函数也是可以用的。假如你想得到frame中各个浮点值的格式化字符串，使用applymap即可：
format = lambda x: '%.2f' % x

In [139]:
frame.applymap(format)

,b,d,e
Utah,0.43,0.22,-0.03
Ohio,1.00,-0.41,0.18
Texas,-1.96,-0.27,0.47
Oregon,1.27,-0.47,-0.21


In [140]:
#之所以叫做applymap，是因为Series有一个用于应用元素级函数的map方法：
frame['e'].map(format)

Utah      -0.03
Ohio       0.18
Texas      0.47
Oregon    -0.21
Name: e, dtype: object

# 排序和排名

In [141]:
#根据条件对数据集排序（sorting）也是一种重要的内置运算。要对行或列索引进行排序（按字典顺序），可使用sort_index方法，它将返回一个已排序的新对象：

In [142]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])

In [143]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

In [144]:
#对于DataFrame，则可以根据任意一个轴上的索引进行排序：

In [145]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                    index=['three', 'one'],
                 columns=['d', 'a', 'b', 'c'])

In [146]:
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [147]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [148]:
frame.sort_index(axis=1)
#axis =0，按照行排序；axis = 1，按照列排序；

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [149]:
#数据默认是按升序排序的，但也可以降序排序：
frame.sort_index(axis=1, ascending=False)#列上的索引

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


In [150]:
#若要按值对Series进行排序，可使用其sort_values方法：
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

In [151]:
#在排序时，任何缺失值默认都会被放到Series的末尾：
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

In [152]:
#当排序一个DataFrame时，你可能希望根据一个或多个列中的值进行排序。将一个或多个列的名字传递给sort_values的by选项即可达到该目的：


####by已删除

In [153]:
###当排序一个DataFrame时，你可能希望根据一个或多个列中的值进行排序。将一个或多个列的名字传递给sort_values的by选项即可达到该目的：
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()
#我们对数据按从小到大排名次：
#-5 是最小的，所以它就排在第一位，即1.0 ；
#同理0 这个数就是第二名，写成 2.0 ；
#同样就可以理解2的写法了；
#我们来到4 这两个数，因为他们重复了，所以我们取它们的排名4.0 和 5.0 的平均值，即写成 4.5；
#这下也不难理解最后同样两个7的写法6.5了。

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [154]:
obj.rank(method='first')
#跟之前的方法差不多，我们先看到那个数字，哪个数字的排名就优先！
#所以这两个7 我们先看到最开头的，它按顺序是排在 第六名的，所以写作 6.0；同理第二个看到的就是第七名，写作 7.0。这样也可以理解两个 4 的排名写法了！

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [155]:
obj.rank(ascending=False, method='max')
#这个方法，所以当两个同样的 7 出现的时候，一个排名写作 1.0 ， 另一个排名写作 2.0 ，我们就取最大的那个值来表示。

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

In [156]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                   'c': [-2, 5, 8, -2.5]})

In [157]:
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [158]:
frame.rank(axis='columns')

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


# 带有重复标签的轴索引

In [159]:
#直到目前为止，我所介绍的所有范例都有着唯一的轴标签（索引值）。虽然许多pandas函数（如reindex）都要求标签唯一，但这并不是强制性的。我们来看看下面这个简单的带有重复索引值的Series：

In [160]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])

In [161]:
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

In [162]:
#索引的is_unique属性可以告诉你它的值是否是唯一的：
obj.index.is_unique

False

In [163]:
#对于带有重复值的索引，数据选取的行为将会有些不同。如果某个索引对应多个值，则返回一个Series；而对应单个值的，则返回一个标量值：

In [164]:
obj['a']

a    0
a    1
dtype: int64

In [165]:
obj['c']

4

In [166]:
#这样会使代码变复杂，因为索引的输出类型会根据标签是否有重复发生变化。

In [167]:
#对DataFrame的行进行索引时也是如此：
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])

In [168]:
df

,0,1,2
a,-0.233126,0.588777,1.525481
a,0.461484,-0.238246,0.094704
b,1.776014,0.583660,-0.447861
b,0.387931,-0.486330,-1.247620


In [169]:
df.loc['b']

,0,1,2
b,1.776014,0.58366,-0.447861
b,0.387931,-0.48633,-1.247620
